In [5]:
rush_hour = "昏峰"    #晨峰或昏峰

import pandas as pd
import numpy as np

df = pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]

# 處理 Excel 中的空值
sheet1 = sheet1.fillna(0)
sheet1 = sheet1.replace('', 0)

ue_results_1 = pd.read_csv(f"104_現況_{rush_hour}/104_現況_{rush_hour}_UE_results.dat", sep="\t")
ue_results_2 = pd.read_csv(f"104_全線拆除_{rush_hour}/104_全線拆除_{rush_hour}_UE_results.dat", sep="\t")
ue_results_3 = pd.read_csv(f"104_部分拆除_{rush_hour}/104_部分拆除_{rush_hour}_UE_results.dat", sep="\t")

# 處理可能的空值
for df_name, df_data in [("ue_results_1", ue_results_1), ("ue_results_2", ue_results_2), ("ue_results_3", ue_results_3)]:
    df_data['tailNode'] = df_data['tailNode'].fillna(0)
    df_data['headNode'] = df_data['headNode'].fillna(0)
    df_data['UE_flow'] = df_data['UE_flow'].fillna(0)

# 安全地建立 link_name
def safe_link_name(tail, head):
    """安全地建立 link_name，處理空值"""
    try:
        if pd.isna(tail) or pd.isna(head) or tail == 0 or head == 0:
            return "0_0"
        return f"{int(tail)}_{int(head)}"
    except (ValueError, TypeError):
        return "0_0"

ue_results_1["link_name"] = ue_results_1.apply(lambda row: safe_link_name(row['tailNode'], row['headNode']), axis=1)
ue_results_2["link_name"] = ue_results_2.apply(lambda row: safe_link_name(row['tailNode'], row['headNode']), axis=1)
ue_results_3["link_name"] = ue_results_3.apply(lambda row: safe_link_name(row['tailNode'], row['headNode']), axis=1)

ue_results_1 = ue_results_1.set_index("link_name")
ue_results_2 = ue_results_2.set_index("link_name")
ue_results_3 = ue_results_3.set_index("link_name")

# 安全地建立 sheet1 的 link_name
def safe_sheet_link_name(a, b):
    """安全地建立 sheet1 的 link_name，處理空值"""
    try:
        if pd.isna(a) or pd.isna(b) or a == 0 or b == 0:
            return "0_0"
        return f"{int(a)}_{int(b)}"
    except (ValueError, TypeError):
        return "0_0"

sheet1["link_name1"] = sheet1.apply(lambda row: safe_sheet_link_name(row['A1'], row['B1']), axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: safe_sheet_link_name(row['A2'], row['B2']), axis=1)

# 安全地合併流量數據，檢查索引是否存在
def safe_add_flow(df, target_link, source_link):
    """安全地合併流量，檢查索引是否存在"""
    try:
        if target_link in df.index and source_link in df.index:
            df.loc[target_link, "UE_flow"] += df.loc[source_link, "UE_flow"]
        else:
            print(f"警告：找不到連結 {target_link} 或 {source_link}")
    except Exception as e:
        print(f"合併流量時發生錯誤：{e}")

# 將承德橋機車道流量合併
safe_add_flow(ue_results_1, '10246_6631', '10245_6631')
safe_add_flow(ue_results_1, '6631_10247', '6631_10248')
safe_add_flow(ue_results_2, '10246_6631', '10245_6631')
safe_add_flow(ue_results_2, '6631_10247', '6631_10248')
safe_add_flow(ue_results_3, '10246_6631', '10245_6631')
safe_add_flow(ue_results_3, '6631_10247', '6631_10248')

# 將中山北路機車道流量合併
safe_add_flow(ue_results_1, '6448_6420', '8371_8376')
safe_add_flow(ue_results_1, '8375_8372', '8374_8373')
safe_add_flow(ue_results_2, '6448_6420', '8371_8376')
safe_add_flow(ue_results_2, '8375_8372', '8374_8373')
safe_add_flow(ue_results_3, '6448_6420', '8371_8376')
safe_add_flow(ue_results_3, '8375_8372', '8374_8373')

# 安全地計算變化百分比
def safe_percentage_change(new_val, old_val):
    """安全地計算百分比變化，處理零值和空值"""
    try:
        if pd.isna(new_val) or pd.isna(old_val) or old_val == 0:
            return 0.0
        return (new_val - old_val) / old_val * 100
    except (ValueError, TypeError, ZeroDivisionError):
        return 0.0

for index, row in sheet1.iterrows():
    try:
        group = row['group']
        if pd.isna(group) or group != 'SL1' or row['link'] == '承德橋(機車道)':
            continue
            
        link_name1 = row['link_name1']
        link_name2 = row['link_name2']
        
        # 檢查連結是否存在於所有結果中
        if (link_name1 not in ue_results_1.index or 
            link_name1 not in ue_results_2.index or 
            link_name1 not in ue_results_3.index or
            link_name2 not in ue_results_1.index or 
            link_name2 not in ue_results_2.index or 
            link_name2 not in ue_results_3.index):
            print(f"警告：連結 {link_name1} 或 {link_name2} 在某些結果中不存在，跳過 {row['link']}")
            continue
        
        # 安全地取得流量數據
        flow_1_s = ue_results_1.loc[link_name1, "UE_flow"]
        flow_2_s = ue_results_2.loc[link_name1, "UE_flow"]
        flow_3_s = ue_results_3.loc[link_name1, "UE_flow"]
        
        flow_1_n = ue_results_1.loc[link_name2, "UE_flow"]
        flow_2_n = ue_results_2.loc[link_name2, "UE_flow"]
        flow_3_n = ue_results_3.loc[link_name2, "UE_flow"]
        
        # 計算變化百分比
        change_2_s = safe_percentage_change(flow_2_s, flow_1_s)
        change_3_s = safe_percentage_change(flow_3_s, flow_1_s)
        change_2_n = safe_percentage_change(flow_2_n, flow_1_n)
        change_3_n = safe_percentage_change(flow_3_n, flow_1_n)
        
        print(f'''{row['link']}
        【南向】  全線拆除: {change_2_s:.2f}%，部分拆除: {change_3_s:.2f}%
        【北向】  全線拆除: {change_2_n:.2f}%，部分拆除: {change_3_n:.2f}%
        ''')
        
    except Exception as e:
        print(f"處理 {row.get('link', 'unknown')} 時發生錯誤：{e}")
        continue

洲美快速道路
        【南向】  全線拆除: 2.47%，部分拆除: 1.37%
        【北向】  全線拆除: -1.62%，部分拆除: -0.30%
        
百齡橋
        【南向】  全線拆除: 4.08%，部分拆除: 2.28%
        【北向】  全線拆除: 7.50%，部分拆除: 1.18%
        
承德橋
        【南向】  全線拆除: 13.89%，部分拆除: -2.36%
        【北向】  全線拆除: 40.03%，部分拆除: 5.44%
        
承德橋(機車專用道)
        【南向】  全線拆除: 8.64%，部分拆除: -13.51%
        【北向】  全線拆除: 7.29%，部分拆除: -6.04%
        
中山橋
        【南向】  全線拆除: 51.48%，部分拆除: 11.27%
        【北向】  全線拆除: 144.01%，部分拆除: 50.31%
        
大直橋
        【南向】  全線拆除: 22.45%，部分拆除: 2.85%
        【北向】  全線拆除: 37.46%，部分拆除: 10.56%
        
高速公路橋
        【南向】  全線拆除: 2.47%，部分拆除: 0.71%
        【北向】  全線拆除: -0.75%，部分拆除: 2.51%
        
民權大橋
        【南向】  全線拆除: 3.16%，部分拆除: 1.23%
        【北向】  全線拆除: 1.98%，部分拆除: 0.17%
        
民權大橋(機車專用道)
        【南向】  全線拆除: 0.80%，部分拆除: 0.39%
        【北向】  全線拆除: 1.84%，部分拆除: -0.19%
        
麥帥二橋
        【南向】  全線拆除: 1.60%，部分拆除: -0.28%
        【北向】  全線拆除: 4.95%，部分拆除: 2.71%
        
麥帥一橋
        【南向】  全線拆除: 8.01%，部分拆除: 1.35%
        【北向】  全線拆除: 2.77%，